<a href="https://colab.research.google.com/github/Malmiquer/Extra-o-de-Dados-em-Arquivos-PDF/blob/main/PFE1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import plotly.express as px

In [ ]:
df = pd.read_excel(r'copie_aqui_o_seu_caminho')

In [ ]:
df.head()
default_test_size = 0.1
seed = 42

In [ ]:
a = df.iloc[:, -1].values
b = df.iloc[:, -1].values

In [ ]:
#Substitua aqui os seus dados
dados= {
    'B_max': df['B dente est'],
    'F': df['Frequency'],
    'KS': df['KS'],
    'Kc1': df['Kc1'],
    'Voltage': df['Voltage'],
    'Potência': df['Potência'],
    'Current': df['Current'],
    'Fator': df['Fator'],
}

In [ ]:
def calcular_perdas_por_histerese(df, B_max, F, KS):
#     """
#     B_max (str): Densidade de fluxo magnético.
#     F (str): Frequência da corrente alternada.
#     KS (str): Constante que depende do material do ferro.

#     """
  return df[KS] * (df[B_max] ** 2) * df[F]

In [ ]:
df = pd.DataFrame(dados)
df['Perdas_por_Histerese'] = calcular_perdas_por_histerese(df, 'B_max', 'F', 'KS')

print(df.head(5))  # Exibe as primeiras 5 linhas do DataFrame

In [ ]:
def perdas_por_correntes_parasitas(df, B_max, F, KS):
#     """
#     B_max (str): Densidade de fluxo magnético.
#     F (str): Frequência da corrente alternada.
#     KS (str): Constante que depende do material do ferro.
#     """
  return df[KS] * (df[B_max] ** 2) * (df[F]**2)


In [ ]:
df = pd.DataFrame(dados)
df['perdas_por_correntes_parasitas'] = perdas_por_correntes_parasitas(df, 'B_max', 'F', 'KS')

print(df.head(5))

In [ ]:
class MotorInducao:
    def __init__(self, Voltage, Current, F, Fator, perdas_por_correntes_parasitas, Perdas_por_Histerese): #O método __init__ é um método conhecido como construtor
        self.Voltage =  Voltage #self.Voltage: Atribui o valor do parâmetro Voltage ao atributo da instância Voltage, de igual modo para os outros dados
        self.Current = Current
        self.F = F
        self.Fator = Fator
        self.perdas_por_correntes_parasitas = perdas_por_correntes_parasitas
        self.Perdas_por_Histerese = Perdas_por_Histerese

    def calcular_perdas_ferro(self):
        perdas_totais = self.perdas_por_correntes_parasitas + self.Perdas_por_Histerese
        return perdas_totais


    def causas_perdas_ferro(self):
        causas = []

        if self.Perdas_por_Histerese > 3% Potência:
            causas.append(f"Perdas por histerese: {self.Perdas_por_Histerese} W - causadas pela magnetização e desmagnetização do núcleo do motor.")

        if self.perdas_por_correntes_parasitas > 3% Potência:
            causas.append(f"Perdas por correntes parasitas: {self.perdas_por_correntes_parasitas} W - geradas pela circulação de correntes em caminhos fechados dentro do núcleo de ferro.")

        if self.F > 60:
            causas.append(f"Perdas por saturação do núcleo: Frequência alta ({self.F} Hz) pode levar à saturação magnética do núcleo.")

        if self.Voltage > 220:
            causas.append(f"Perdas por acoplamento magnético: Tensão alta ({self.Voltage} V) pode causar vazamentos de fluxo magnético entre as partes do motor.")

        return causas

In [ ]:
df = pd.DataFrame(dados)
df['Perdas_por_Histerese'] = calcular_perdas_por_histerese(df, 'B_max', 'F', 'KS')
df['perdas_por_correntes_parasitas'] = perdas_por_correntes_parasitas(df, 'B_max', 'F', 'KS')


In [ ]:
 # Exemplo de uso
Voltage = df['Voltage'].iloc[0]
current = df['Current'].iloc[0]
frequency = df['F'].iloc[0]
fator = df['Fator'].iloc[0]
Potência = df['Potência'].iloc[0]

In [ ]:
Perdas_por_correntes_parasitas = df['perdas_por_correntes_parasitas'].iloc[0]
Perdas_por_Histerese = df['Perdas_por_Histerese'].iloc[0]

In [ ]:
motor = MotorInducao(Voltage, current, frequency, fator, Perdas_por_correntes_parasitas, Perdas_por_Histerese)
perdas_totais = motor.calcular_perdas_ferro()
causas = motor.causas_perdas_ferro()

In [ ]:
print(f"Perdas totais de ferro: {perdas_totais} W")
print("Principais causas das perdas de ferro:")
for causa in causas:
    print(f"- {causa}")

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler #vai fazer as standarlização dos dados
from sklearn.model_selection import train_test_split #vai separar os dados de treinos dos dados reais
from sklearn.model_selection import GridSearchCV # vai encontrar os melhores hiper parâmetros do nosso sistema de regressão

from sklearn.neighbors import KNeighborsRegressor #metodo de regressão knn

In [ ]:
a_train, a_test, b_train, b_test = train_test_split(a, b, test_size=default_test_size, random_state=seed)

In [ ]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', KNeighborsRegressor())
    ])

In [ ]:
hyperparameters = {'regressor__n_neighbors': [2,3,5,10], #tem o andscor(__) duas vezes para permitir procurar os vizinhos, ou seja determina a quantidade de vizinhos
                   'regressor__weights': ['uniform','distance'],
                   'regressor__algorithm': ['auto','ball_tree', 'kd_tree', 'brute'],
                  }

In [ ]:
grid_search = GridSearchCV(pipe,
                           param_grid=hyperparameters,
                           return_train_score=True,
                           scoring='neg_root_mean_squared_error',
                           n_jobs=-2,
                           cv = 5)

In [ ]:
a_train_reshaped = a_train.reshape(-1, 1)
grid_search.fit(a_train_reshaped, b_train)

In [ ]:
# Print Best Hyperparameters
cv_best_params = grid_search.best_params_
print('\n Best hyperparameters:')
print(cv_best_params)

In [ ]:
pipe.set_params(**cv_best_params)

In [ ]:
pipe.get_params()

In [ ]:
pipe.fit(a_train_reshaped, b_train)

In [ ]:
b_test_pred = pipe.predict(a_test.reshape(-1, 1))

In [ ]:
a_test

In [ ]:
##Calculando r^2
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

In [ ]:
# Análise dos erros das previsões
rmse_test = math.sqrt(mean_squared_error(b_test, b_test_pred))
mae_test = mean_absolute_error(b_test, b_test_pred)
mape_test = mean_absolute_percentage_error(b_test, b_test_pred)
r2_test = r2_score(b_test, b_test_pred)

df_metricas = pd.DataFrame(data={'RSME':[rmse_test], 'MAE':[mae_test], 'MAPE':[mape_test],'R²':[r2_test]})
df_metricas

In [ ]:
## Comparando os valores Reais dos valores Preditos
b_pred = pd.DataFrame(data=pipe.predict(a_test.reshape(-1, 1)), columns=['Predicted Values'])
b_real = pd.DataFrame(data=b_test, columns=['Real Values'])

In [ ]:
# Preparation of the comparative DataFrame between the prediction and the actual value
df_comparison = pd.concat([b_real, b_pred],axis=1)
df_comparison.columns = ['Real_Data','Predicted_Value']
df_comparison['Percentage_difference'] = 100*(df_comparison['Predicted_Value'] - df_comparison['Real_Data'])/df_comparison['Real_Data']
df_comparison['Average'] = df_comparison['Real_Data'].mean()
df_comparison['Q1'] = df_comparison['Real_Data'].quantile(0.25)
df_comparison['Q3'] = df_comparison['Real_Data'].quantile(0.75)
df_comparison['USL'] = df_comparison['Real_Data'].mean() + 2*df_comparison['Real_Data'].std()
df_comparison['LSL'] = df_comparison['Real_Data'].mean() - 2*df_comparison['Real_Data'].std()

df_comparison.sort_index(inplace=True)
df_comparison

In [ ]:
# Graphic visualization of predictions by real values
plt.figure(figsize=(25,10))
plt.title('Real Value vs Predicted Value', fontsize=25)
plt.plot(df_comparison.index, df_comparison['Real_Data'], label = 'Real', marker='D', markersize=10, linewidth=0)
plt.plot(df_comparison.index, df_comparison['Predicted_Value'], label = 'Predicted', c='r', linewidth=1.5)
plt.plot(df_comparison.index, df_comparison['Average'], label = 'Mean', linestyle='dashed', c='yellow')
plt.plot(df_comparison.index, df_comparison['Q1'], label = 'Q1', linestyle='dashed',c='g')
plt.plot(df_comparison.index, df_comparison['Q3'], label = 'Q3', linestyle='dashed',c='g')

plt.plot(df_comparison.index, df_comparison['USL'], label = 'USL', linestyle='dashed',c='r')
plt.plot(df_comparison.index, df_comparison['LSL'], label = 'LSL', linestyle='dashed',c='r')

plt.legend(loc='best')
plt.legend(fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.show()

In [ ]:
# Graphic visualization of predictions by real values
plt.figure(figsize=(25,10))
plt.title('Real Value vs Predicted Value',fontsize=25)
plt.scatter(df_comparison['Real_Data'], df_comparison['Predicted_Value'], s=100)
plt.plot(df_comparison['Real_Data'],df_comparison['Real_Data'],c='r')

plt.xlabel('Real', fontsize=25)
plt.ylabel('Predicted', fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.show()

In [ ]:
# Comparação entre as perdas
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Verificar onde os valores em 'Perdas_por_Histerese' são maiores que em 'perdas_por_correntes_parasitas'
# df['coluna_comparacao'] = df['Perdas_por_Histerese'] > df['perdas_por_correntes_parasitas']
df['coluna_comparacao'] = df['Potência'] > df['perdas_por_correntes_parasitas']

In [ ]:
# Gráfico de barras
# df[['Perdas_por_Histerese', 'perdas_por_correntes_parasitas']].plot(kind='bar', figsize=(20, 6))
df[['Potência', 'perdas_por_correntes_parasitas']].plot(kind='bar', figsize=(20, 6))
# plt.title('Comparação entre Perdas_por_Histerese e perdas_por_correntes_parasitas')
plt.title('Comparação entre Potência e perdas_por_correntes_parasitas')
plt.xlabel('Índice')
plt.ylabel('Valores')

plt.show()

In [ ]:
df['coluna_comparacao'] = df['Potência'] > df['Perdas_por_Histerese']
df[['Potência', 'Perdas_por_Histerese']].plot(kind='bar', figsize=(20, 6))
plt.title('Comparação entre Potência e Perdas_por_Histerese')
plt.xlabel('Índice')
plt.ylabel('Valores')

plt.show()

In [ ]:
# Função para anotar as diferenças e causas
def annotate_differences_and_causes(ax, df):
    for i in df.index:
        if df['Perdas_por_Histerese'][i] != df['perdas_por_correntes_parasitas'][i]:
            diff = df['Perdas_por_Histerese'][i] - df['perdas_por_correntes_parasitas'][i]
            cause = 'Causa: Ajuste de parâmetros' if diff > 0 else 'Causa: Variação de material'
            ax.annotate(f'Dif.: {diff:.2f}\n{cause}',
                        xy=(i, max(df['Perdas_por_Histerese'][i], df['perdas_por_correntes_parasitas'][i])),
                        xytext=(i, max(df['Perdas_por_Histerese'][i], df['perdas_por_correntes_parasitas'][i]) + 0.5),
                        arrowprops=dict(facecolor='black', shrink=0.05), fontsize=8)

    # Gráfico de comparação
plt.figure(figsize=(10, 6))

In [ ]:
# Plotando as três séries de dados
plt.plot(df['Perdas_por_Histerese'], label='Perdas_por_Histerese', marker='o')
plt.plot(df['perdas_por_correntes_parasitas'], label='perdas_por_correntes_parasitas', marker='x')
#plt.plot(df['Perda Newton com fator'], label='Perda Newton com fator', marker='s')

# Preenchendo entre as séries de dados
plt.fill_between(df.index, df['Perdas_por_Histerese'], df['perdas_por_correntes_parasitas'],
                 where=(df['Perdas_por_Histerese'] > df['perdas_por_correntes_parasitas']),
                 facecolor='green', alpha=0.3, interpolate=True)
plt.fill_between(df.index, df['Perdas_por_Histerese'], df['perdas_por_correntes_parasitas'],
                 where=(df['Perdas_por_Histerese'] <= df['perdas_por_correntes_parasitas']),
                 facecolor='red', alpha=0.3, interpolate=True)

# Configurações do gráfico
plt.title('Comparação Gráfica entre Perdas_por_Histerese e perdas_por_correntes_parasitas')
plt.xlabel('Índice')
plt.ylabel('Valores')
plt.legend()
plt.show()